In [25]:
import torchvision.transforms as transforms
from torchvision.utils import save_image, make_grid

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

from my_resunet import unet
from torch_unet import UNet
import data

import os
import numpy as np
import math
import itertools
from imgaug import augmenters as iaa

In [26]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [27]:
net = unet().to(device)
net.load_state_dict(torch.load("saved_mymodel_bce/model_tooth_399.pth"))

<All keys matched successfully>

In [28]:
root = "../../dataset/opg-decomp-pilot/"

In [29]:
tfs = transforms.Compose([
    iaa.Sequential([
        #iaa.Multiply(mul=0.00392156862),
        #iaa.LinearContrast(alpha=(0.5, 1.5)),
        #iaa.Multiply(mul=(0.7, 1.3)),
        #iaa.Affine(scale=(0.8, 1.2), rotate=(-10, 10), shear=(-10, 10)),
        #iaa.ElasticTransformation(alpha=(0, 100), sigma=10),
        #iaa.GaussianBlur(sigma=(0.0, 5.0)),
        #iaa.Cutout(size=0.1),
        iaa.Resize(size={'height': 192, 'width': 512}),
        #iaa.PadToFixedSize(height=192, width=512),
        #iaa.CropToFixedSize(height=192, width=512),
    ]).augment_image,
    transforms.ToTensor()
])

In [30]:
custom_ds = data.CustomDataset(root, transform=tfs)
custom_dl = DataLoader(custom_ds, batch_size=1,
            num_workers=34, pin_memory=True)

In [34]:
sigmoid = nn.Sigmoid()
for i, batch in enumerate(custom_dl):
    net.eval()
    input = batch["row"].to(device)
    mask = batch["tooth"].to(device)
    output_prob = sigmoid(net(input))
    #output_prob_ths = (output_prob>0.5)*1.0

    img_input = make_grid(input)
    img_output_prob = make_grid(output_prob)
    #img_output_prob_ths = make_grid(output_prob_ths)
    img_mask = make_grid(mask)


    image_grid = torch.cat((img_input, img_output_prob, img_mask), 1)
    save_image(image_grid, "test_img/%s.png" %(i), normalize=False)
    break
